In [18]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', 1000000)
pd.set_option('display.max_rows', 1000000)

In [19]:
data = pd.read_csv('Amazon.csv')
print(data.shape)
data.Summary = data.Summary.fillna('0')
data.ProfileName = data.ProfileName.fillna(' ')
data.head(5)

(455000, 13)


,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,helpScore,helpful
0,138806,138807,B000E63LME,A1CQGW1AOD0LF2,"Alena K. ""Alena""",1,2,2,1294185600,Not as pictured.,I was looking forward to try cranberry apple f...,0.5,False
1,469680,469681,B004ZIH4KM,A37S7U1OX2MCWI,Becky Cole,0,0,5,1349740800,seeds,"TY for everything. The seeds arrived quickly,...",NaN,False
2,238202,238203,B003ZXE9QA,A2OM6G73E64EQ9,jeff,0,0,5,1329264000,I'm addicted!,I've finally found the best cereal in the worl...,NaN,False
3,485307,485308,B001RVFERK,A25W349EE97NBK,Tangent4,1,1,4,1248307200,I wanted to love these...,I originally bought these chips because I'd he...,1.0,False
4,375283,375284,B000OQZNTS,A3CPPW0HUC07YS,Amy Nicolai,0,0,5,1333238400,Excellent chamomile tea,"Really excellent tea, flowers are visible in t...",NaN,False


In [20]:
data['hasSC'] = data['Text'].str.contains(';')
data['hasbroken'] = data['Text'].str.contains('broken')
data['hasdidnt'] = data['Text'].str.contains('didn\'t')
data['hasperfect'] = data['Text'].str.contains('perfect')
data['haslove'] = data['Text'].str.contains('love')
data['hasEP'] = data['Text'].str.contains('!')
data['hasdash'] = data['Text'].str.contains('-')
data['hasChocolate'] = data['Text'].str.contains('chocolate')
data['hasCoffee'] = data['Text'].str.contains('coffee')
data['hasRec'] = data['Text'].str.contains('recommend')
data['hasTerrible'] = data['Text'].str.contains('terrible')



data['sumEP'] = data['Summary'].str.contains('!')
data['sumPeriod'] = data['Summary'].str.contains('.')
data['nameSpace'] = data['ProfileName'].str.contains(' ')

data['sumLen'] = data['Summary'].str.len()






data['reviewLen'] = data['Text'].str.len()
data['duplicateIds'] = data.UserId.duplicated()
data['nameLen'] = data['ProfileName'].str.len()

data['sumtexdif'] = data['Text'].str.len() - data['Summary'].str.len()
data.sumtexdif[data.sumtexdif < 0] = 0

data['sumComma'] = data['Summary'].str.contains(',')
data['sumColon'] = data['Summary'].str.contains(':')
data['sumSC'] = data['Summary'].str.contains(';')
data['sumDash'] = data['Summary'].str.contains('-')

data['hasComma'] = data['Text'].str.contains(',')
data['hasColon'] = data['Text'].str.contains(':')

data['ProductCounts'] = data.groupby('ProductId')['ProductId'].transform('count')
data['UserCounts'] = data.groupby('UserId')['UserId'].transform('count')




/Users/ianssmith/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
XScore = data.iloc[:, 7].values.reshape(data.shape[0], 1)

hasSC = data.iloc[:, 13].values.reshape(data.shape[0], 1)
hasbroken = data.iloc[:, 14].values.reshape(data.shape[0], 1)
hasdidnt = data.iloc[:, 15].values.reshape(data.shape[0], 1)
hasperfect = data.iloc[:, 16].values.reshape(data.shape[0], 1)
haslove = data.iloc[:, 17].values.reshape(data.shape[0], 1)
hasEP = data.iloc[:, 18].values.reshape(data.shape[0], 1)
hasdash = data.iloc[:, 19].values.reshape(data.shape[0], 1)
hasChocolate = data.iloc[:, 20].values.reshape(data.shape[0],1)

hasCoffee = data.iloc[:, 21].values.reshape(data.shape[0],1)

hasRec = data.iloc[:, 22].values.reshape(data.shape[0],1)
hasTerrible = data.iloc[:, 23].values.reshape(data.shape[0],1)

sumEP = data.iloc[:, 24].values.reshape(data.shape[0],1)
sumPeriod = data.iloc[:, 25].values.reshape(data.shape[0],1)
nameSpace = data.iloc[:, 26].values.reshape(data.shape[0],1)

sumLen = data.iloc[:, 27].values.reshape(data.shape[0],1)




XreviewLen = data.iloc[:, 28].values.reshape(data.shape[0], 1)
XIddups = data.iloc[:, 29].values.reshape(data.shape[0], 1)

nameLen = data.iloc[:, 30].values.reshape(data.shape[0], 1)

sumtexdif = data.iloc[:, 31].values.reshape(data.shape[0], 1)

sumComma = data.iloc[:, 32].values.reshape(data.shape[0],1)
sumColon = data.iloc[:, 33].values.reshape(data.shape[0],1)
sumSC = data.iloc[:, 34].values.reshape(data.shape[0],1)
sumDash = data.iloc[:, 35].values.reshape(data.shape[0],1)
hasComma = data.iloc[:, 36].values.reshape(data.shape[0],1)
hasColon = data.iloc[:, 37].values.reshape(data.shape[0],1)
ProductCounts = data.iloc[:, 38].values.reshape(data.shape[0],1)
UserCounts = data.iloc[:, 39].values.reshape(data.shape[0],1)





Xtoadd = np.concatenate(( hasSC, hasbroken, hasdidnt, hasperfect, 
                         haslove, hasEP, hasdash, XScore, XreviewLen,
                         XIddups, hasChocolate, hasCoffee, hasRec,
                         hasTerrible, sumtexdif, sumEP, sumPeriod,
                         sumLen, nameLen, nameSpace, sumComma, sumColon,
                         sumSC, sumDash, hasComma, hasColon, ProductCounts, UserCounts), axis=1)
#type(Xtoadd)
#data.iloc[100000:101000, 27]
Xtoadd.dtype


dtype('int64')

In [22]:
def print_results():
    print('Error rate on training set: ')
    print((y_train != y_pred).sum() / X_train.shape[0])
    print('Accuracy rate on training set: ')
    print(1 - (y_train != y_pred).sum() / X_train.shape[0])
    print('True positive rate on training tet:')
    print(((y_train==True) & (y_pred==True)).sum() / y_train.sum())
    print('**************')
    print('Error rate on test set: ')
    print((y_test != y_pred_test).sum() / X_test.shape[0])
    print('Accuracy rate on test set: ')
    print(1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    print('True positive rate on test set')
    print(((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    print('True negative rate on test set')
    print(((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))

In [23]:
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=2 ** 17, non_negative=True) ##use n_features= to restrict features
X = hv.transform(data.Text)
Xsum = hv.transform(data.Summary)
Xname = hv.transform(data.ProfileName)
Xuid = hv.transform(data.UserId)
Xpid = hv.transform(data.ProductId)
 
type(X)

scipy.sparse.csr.csr_matrix

In [26]:
import nltk.stem
from sklearn.feature_extraction.text import CountVectorizer
english_stemmer = nltk.stem.SnowballStemmer('english')

In [27]:
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

In [28]:
vectorizer = StemmedCountVectorizer(min_df=1,
stop_words='english')

In [29]:
Xstemmer = vectorizer.fit_transform(data.Text)

In [30]:
from scipy.sparse import csr_matrix, hstack
XtoaddSparse = csr_matrix(Xtoadd)

In [31]:
Xfinal = hstack([X, Xsum, Xname, Xpid, Xuid, Xstemmer, XtoaddSparse])
X = csr_matrix(Xfinal)

In [32]:
y = data.iloc[:, 12].values

In [33]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [34]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [41]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results()

Error rate on training set: 
0.0729293563579
Accuracy rate on training set: 
0.927070643642
True positive rate on training tet:
0.0
**************
Error rate on test set: 
0.0733113553114
Accuracy rate on test set: 
0.926688644689
True positive rate on test set
0.0
True negative rate on test set
1.0


In [44]:
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results()

Error rate on training set: 
0.000210361067504
Accuracy rate on training set: 
0.999789638932
True positive rate on training tet:
0.998622352333
**************
Error rate on test set: 
0.0930769230769
Accuracy rate on test set: 
0.906923076923
True positive rate on test set
0.385230338763
True negative rate on test set
0.948194761765


In [43]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.fit(X_train, y_train).predict(X_train)
y_pred_test = clf.predict(X_test)
print_results()

Error rate on training set: 
0.344260596546
Accuracy rate on training set: 
0.655739403454
True positive rate on training tet:
0.587911141726
**************
Error rate on test set: 
0.355985347985
Accuracy rate on test set: 
0.644014652015
True positive rate on test set
0.574497851504
True negative rate on test set
0.649514202367


In [46]:
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results()

Error rate on training set: 
0.00469387755102
Accuracy rate on training set: 
0.995306122449
True positive rate on training tet:
0.965860168762
**************
Error rate on test set: 
0.0948424908425
Accuracy rate on test set: 
0.905157509158
True positive rate on test set
0.382132507245
True negative rate on test set
0.946534590847


In [47]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
y_pred = lr.fit(X_train, y_train).predict(X_train)
y_pred_test = lr.predict(X_test)
print_results()

Error rate on training set: 
0.0677205651491
Accuracy rate on training set: 
0.932279434851
True positive rate on training tet:
0.0951007404856
**************
Error rate on test set: 
0.0718315018315
Accuracy rate on test set: 
0.928168498168
True positive rate on test set
0.0668532027581
True negative rate on test set
0.9963080961
